<a href="https://colab.research.google.com/github/chirag-sharma-00/cs182-cv-project/blob/main/Copy_of_project_notebook_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm
import os
import time
import copy
from __future__ import print_function
from __future__ import division
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from PIL import Image
from google.colab import drive

In [ ]:
!rm -rf sample_data


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/tiny-imagenet-200.zip

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_446.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_153.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_295.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_411.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_104.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_387.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_18.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_368.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_407.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_112.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_391.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/images/n01984695_450.JPEG  
  inflating: tiny-imagenet-200/train/n01984695/ima

In [ ]:
data_dir = pathlib.Path('tiny-imagenet-200/')

In [ ]:
#image_count = len(list(data_dir.glob('**/*.JPEG')))
!rm tiny-imagenet-200/.DS_Store
!rm tiny-imagenet-200/train/.DS_Store
!rm tiny-imagenet-200/val/.DS_Store
!rm tiny-imagenet-200/test/.DS_Store

rm: cannot remove 'tiny-imagenet-200/train/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/val/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/test/.DS_Store': No such file or directory


## Preliminary Analysis

In [ ]:
images = pd.read_csv("tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [ ]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
CLASS_NAMES.sort()
print(len(CLASS_NAMES)) #should be 200
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200


200

## Create augmented validation data folder

In [ ]:
import data_augmentation as aug
import os

In [ ]:
#os.mkdir("tiny-imagenet-200/val/augmented_images")

prev_annotations = pd.read_csv("tiny-imagenet-200/val/val_annotations.txt", 
                              sep='\t', names=["Filename", "Class", "BB1", 
                                               "BB2", "BB3", "BB4"])
prev_annotations.drop(["BB1", "BB2", "BB3", "BB4"], axis=1, inplace=True)
new_annotations = pd.DataFrame(columns=["Filename", "Class"])

for img_file in os.listdir("tiny-imagenet-200/val/images"):
  path = "tiny-imagenet-200/val/images" + "/" + img_file
  augmented_imgs = aug.augmented_data_from_path(path)
  for i, a in enumerate(augmented_imgs):
    im = Image.fromarray(a)
    prefix = img_file.split(".")[0]
    im.save("tiny-imagenet-200/val/augmented_images/" + prefix + "_" + str(i) + 
            ".JPEG")
    new_annotations = new_annotations.append({"Filename" : prefix + "_" + str(i) + ".JPEG", 
                            "Class" : prev_annotations.loc[
                              prev_annotations["Filename"] == prefix + ".JPEG",
                              "Class"
                            ].item()}, ignore_index=True)
new_annotations.to_csv("tiny-imagenet-200/val/augmented_val_annotations.txt", 
                       sep='\t', index=False, header=False)

In [ ]:
!zip -r /content/augmented_images.zip /content/tiny-imagenet-200/val/augmented_images/

	zip warning: name not matched: /content/tiny-imagenet-200/val/augmented_images/

zip error: Nothing to do! (try: zip -r /content/augmented_images.zip . -i /content/tiny-imagenet-200/val/augmented_images/)


## Feature extraction/fine tuning model training code

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=1, is_inception=True):
    if torch.cuda.is_available():
      model = model.cuda()
      
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training (no val for now)
        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # move to gpu
                inputs = inputs.to(device)
                labels.data = labels.data.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    if torch.cuda.is_available():
      model = model.cuda()

    accuracies = []
    model.eval()
    
    for phase in tqdm.tqdm(['train', 'val']): 
        counter = 0
        running_corrects = 0
        running_total = 0

        for inputs, labels in dataloaders[phase]: 
            inputs = inputs.to(device)
            labels.data = labels.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            counter += 1
            
            running_corrects += torch.sum(preds == labels.data)
            running_total += len(preds)
            
        phase_acc = running_corrects / running_total
        print(phase_acc)
        accuracies.append(phase_acc)

    return accuracies
            

In [ ]:
# Custom dataloader based on https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
class ValidationDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep='\t', names=['image', 'label', 'x1', 'y1', 'x2', 'y2'])
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = Image.open(img_path)
    image = image.convert('RGB')
    label = self.img_labels.iloc[idx, 1]
    label = np.where(label==CLASS_NAMES)[0][0]
    if self.transform:
        image = self.transform(image)
    return image, label

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Testing Inception v3 out of the box

In [ ]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True
CUDA_LAUNCH_BLOCKING=1

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [ ]:
if torch.cuda.is_available():
  model.cuda()

In [ ]:
#inception expects input size 3*299*299
input_size = 299

### Feature extracting model performance on original validation data

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), 
                                  data_transform),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'val_annotations.txt'), 
                                      os.path.join(data_dir, 'val', 'images'), data_transform),
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

In [ ]:
num_classes = 200
input_size = 299

model_fe = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model_fe, feature_extract)
# Handle the auxilary net
num_ftrs = model_fe.AuxLogits.fc.in_features
model_fe.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net
num_ftrs = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_ftrs,num_classes)

In [ ]:
# Send the model to GPU
model_fe = model_fe.to(device)

# Gather the parameters to be optimized/updated in this run. We will only update 
# the parameters that we have just initialized, i.e. the parameters with 
# requires_grad is True.
print("Params to learn:")
params_to_update = []
for name, param in model_fe.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [ ]:
optimizer_fe = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_fe, hist = train_model(model_fe, dataloaders_dict, criterion, 
                             optimizer_fe, num_epochs=num_epochs, 
                             is_inception=True)

In [ ]:
accuracies = predict(dataloaders_dict, model_fe)

In [ ]:
for i, key in enumerate(image_datasets.keys()):
  print(key + " accuracy = ", accuracies[i])

In [ ]:
torch.save(model_fe.state_dict(), 'cs182_project_models/oob-inception.pt')

### Feature extracting model performance on augmented validation data

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), 
                                  data_transform),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'augmented_val_annotations.txt'), 
                                      os.path.join(data_dir, 'val', 'augmented_val_images'), data_transform),
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

In [ ]:
accuracies = predict(dataloaders_dict, model_fe)

In [ ]:
for i, key in enumerate(image_datasets.keys()):
  print(key + " accuracy = ", accuracies[i])

## Fine-tuning the out of the box Inception v3 model

In [ ]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True
CUDA_LAUNCH_BLOCKING=1

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [ ]:
if torch.cuda.is_available():
  model.cuda()

In [ ]:
#inception expects input size 3*299*299
input_size = 299

### Fine-tuned model performance on original validation data

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), 
                                  data_transform),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'val_annotations.txt'), 
                                      os.path.join(data_dir, 'val', 'images'), data_transform),
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

NameError: ignored

In [ ]:
num_classes = 200
input_size = 299

model_fe = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model_fe, not feature_extract)
# Handle the auxilary net
num_ftrs = model_fe.AuxLogits.fc.in_features
model_fe.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net
num_ftrs = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_ftrs,num_classes)

In [ ]:
# Send the model to GPU
model_fe = model_fe.to(device)

# Gather the parameters to be optimized/updated in this run. We will only update 
# the parameters that we have just initialized, i.e. the parameters with 
# requires_grad is True.
print("Params to learn:")
params_to_update = []
for name, param in model_fe.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
optimizer_fe = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_fe, hist = train_model(model_fe, dataloaders_dict, criterion, 
                             optimizer_fe, num_epochs=num_epochs, 
                             is_inception=True)

Epoch 0/2
----------
train Loss: 3.3031 Acc: 0.5644
val Loss: 0.9213 Acc: 0.7682

Epoch 1/2
----------
train Loss: 1.3897 Acc: 0.7857
val Loss: 0.8355 Acc: 0.7884

Epoch 2/2
----------
train Loss: 0.9278 Acc: 0.8555
val Loss: 0.8725 Acc: 0.7807

Training complete in 45m 4s
Best val Acc: 0.788400


In [ ]:
accuracies = predict(dataloaders_dict, model_fe)

 50%|█████     | 1/2 [05:11<05:11, 311.14s/it]

tensor(0.8961, device='cuda:0')


100%|██████████| 2/2 [05:42<00:00, 171.38s/it]

tensor(0.7884, device='cuda:0')


In [ ]:
for i, key in enumerate(image_datasets.keys()):
  print(key + " accuracy = ", accuracies[i])

train accuracy =  tensor(0.8961, device='cuda:0')
val accuracy =  tensor(0.7884, device='cuda:0')


In [ ]:
torch.save(model_fe.state_dict(), '/content/drive/MyDrive/cs182_project_models/fine-tuned-inception.pt')

### Fine-tuned model performance on augmented validation data

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), 
                                  data_transform),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'augmented_val_annotations.txt'), 
                                      os.path.join(data_dir, 'val', 'augmented_val_images'), data_transform),
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

In [ ]:
accuracies = predict(dataloaders_dict, model_fe)



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [05:08<05:08, 308.68s/it]

tensor(0.8961, device='cuda:0')




100%|██████████| 2/2 [05:40<00:00, 170.02s/it]

tensor(0.6098, device='cuda:0')


In [ ]:
for i, key in enumerate(image_datasets.keys()):
  print(key + " accuracy = ", accuracies[i])

train accuracy =  tensor(0.8961, device='cuda:0')
val accuracy =  tensor(0.6098, device='cuda:0')


##Adversarial training

In [ ]:
#load finetuned model
model.load_state_dict(torch.load('drive/MyDrive/cs182_project_models/fine-tuned-augmented-train-inception.pt'))
model.eval()

NameError: ignored

In [ ]:
!pip install torchattacks

In [ ]:
import torchattacks
atks = [torchattacks.FGSM(model, eps=8/255),
        torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=7),
        torchattacks.Square(model, eps=8/255),
       ]

In [ ]:
def train_model_adv(model, dataloaders, criterion, optimizer, num_epochs=3, is_inception=True):
    """
    Takes a pretrained model and trains it using a unique adversarial attack for each epoch
    """
    if torch.cuda.is_available():
      model = model.cuda()
      
    since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        atk = atks[epoch]

        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # move to gpu
                inputs = inputs.to(device)
                #adv attack
                inputs = atk(inputs, labels)

                labels.data = labels.data.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)

                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_fe = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
model_adv, hist = train_model_adv(model_fe, dataloaders_dict, criterion, 
                             optimizer_fe, num_epochs=3, 
                             is_inception=True)

###Test on Imagenet-a, Imagenet-o data

In [ ]:
!unzip drive/MyDrive/182-cv-project/data/imagenet-a.zip
imagenet_a_path = pathlib.Path('imagenet-a/')
!unzip drive/MyDrive/182-cv-project/data/imagenet-o.zip
imagenet_o_path = pathlib.Path('imagenet-o/')

In [ ]:
class AdversarialDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep='\t', names=['image', 'label', 'x1', 'y1', 'x2', 'y2'])
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = Image.open(img_path)
    image = image.convert('RGB')
    label = self.img_labels.iloc[idx, 1]
    label = np.where(label==CLASS_NAMES)[0][0]
    if self.transform:
        image = self.transform(image)
    return image, label

##Adversarial training with stability objective

In [ ]:
!pip install torchattacks

In [ ]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True
CUDA_LAUNCH_BLOCKING=1

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [ ]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [ ]:
if torch.cuda.is_available():
  model.cuda()

In [ ]:
#load finetuned model
model.load_state_dict(torch.load('drive/MyDrive/chet_models/fine-tuned-augmented-train-inception.pt'))
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
num_classes = 200
input_size = 299

model_fe = model
set_parameter_requires_grad(model_fe, not feature_extract)
# Handle the auxilary net
num_ftrs = model_fe.AuxLogits.fc.in_features
model_fe.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net
num_ftrs = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_ftrs,num_classes)

In [ ]:
# Send the model to GPU
model_fe = model_fe.to(device)

# Gather the parameters to be optimized/updated in this run. We will only update 
# the parameters that we have just initialized, i.e. the parameters with 
# requires_grad is True.
print("Params to learn:")
params_to_update = []
for name, param in model_fe.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 Conv2d_1a_3x3.conv.weight
	 Conv2d_1a_3x3.bn.weight
	 Conv2d_1a_3x3.bn.bias
	 Conv2d_2a_3x3.conv.weight
	 Conv2d_2a_3x3.bn.weight
	 Conv2d_2a_3x3.bn.bias
	 Conv2d_2b_3x3.conv.weight
	 Conv2d_2b_3x3.bn.weight
	 Conv2d_2b_3x3.bn.bias
	 Conv2d_3b_1x1.conv.weight
	 Conv2d_3b_1x1.bn.weight
	 Conv2d_3b_1x1.bn.bias
	 Conv2d_4a_3x3.conv.weight
	 Conv2d_4a_3x3.bn.weight
	 Conv2d_4a_3x3.bn.bias
	 Mixed_5b.branch1x1.conv.weight
	 Mixed_5b.branch1x1.bn.weight
	 Mixed_5b.branch1x1.bn.bias
	 Mixed_5b.branch5x5_1.conv.weight
	 Mixed_5b.branch5x5_1.bn.weight
	 Mixed_5b.branch5x5_1.bn.bias
	 Mixed_5b.branch5x5_2.conv.weight
	 Mixed_5b.branch5x5_2.bn.weight
	 Mixed_5b.branch5x5_2.bn.bias
	 Mixed_5b.branch3x3dbl_1.conv.weight
	 Mixed_5b.branch3x3dbl_1.bn.weight
	 Mixed_5b.branch3x3dbl_1.bn.bias
	 Mixed_5b.branch3x3dbl_2.conv.weight
	 Mixed_5b.branch3x3dbl_2.bn.weight
	 Mixed_5b.branch3x3dbl_2.bn.bias
	 Mixed_5b.branch3x3dbl_3.conv.weight
	 Mixed_5b.branch3x3dbl_3.bn.weight
	 Mixed_5b.b

In [ ]:
import torchattacks
atks = [torchattacks.FGSM(model, eps=8/255),
        torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=7),
       ]

In [ ]:
def train_model_adv(model, dataloaders, criterion, optimizer, num_epochs=3, 
                    is_inception=True, stability_param=0.1):
    """
    Takes a pretrained model and trains it using a unique adversarial attack for each epoch
    """
    if torch.cuda.is_available():
      model = model.cuda()
      
    since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        atk = np.random.choice(atks)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # move to gpu
                inputs = inputs.to(device)
                #adv attack
                adv_inputs = atk(inputs, labels)

                labels.data = labels.data.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)

                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss_0 = loss1 + 0.4*loss2

                        adv_outputs, adv_aux_outputs = model(adv_inputs)
                        outputs = nn.LogSoftmax(dim=1)(outputs)
                        adv_outputs = nn.LogSoftmax(dim=1)(adv_outputs)
                        aux_outputs = nn.LogSoftmax(dim=1)(aux_outputs)
                        adv_aux_outputs = nn.LogSoftmax(dim=1)(adv_aux_outputs)

                        stability_loss = nn.KLDivLoss(log_target=True)
                        loss1 = stability_loss(outputs, adv_outputs)
                        loss2 = stability_loss(aux_outputs, adv_aux_outputs)
                        loss_s = loss1 + 0.4*loss2

                        loss = loss_0 + stability_param * loss_s
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_fe = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), 
                                  data_transform),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'augmented_val_annotations.txt'), 
                                      os.path.join(data_dir, 'val', 'augmented_val_images'), data_transform),
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

In [ ]:
model_adv, hist = train_model_adv(model_fe, dataloaders_dict, criterion, 
                             optimizer_fe, num_epochs=3, 
                             is_inception=True)

Epoch 0/2
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2611: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


train Loss: 2.2110 Acc: 0.7418
val Loss: 2.5617 Acc: 0.4690
Epoch 1/2
----------
train Loss: 0.9877 Acc: 0.8552
val Loss: 3.1767 Acc: 0.3944
Epoch 2/2
----------
train Loss: 0.6753 Acc: 0.8991
val Loss: 3.0035 Acc: 0.4322
Training complete in 127m 24s
Best val Acc: 0.469000


In [ ]:
accuracies = predict(dataloaders_dict, model_fe)

 50%|█████     | 1/2 [05:17<05:17, 317.20s/it]

tensor(0.6888, device='cuda:0')


100%|██████████| 2/2 [05:49<00:00, 174.80s/it]

tensor(0.4690, device='cuda:0')


In [ ]:
for i, key in enumerate(image_datasets.keys()):
  print(key + " accuracy = ", accuracies[i])

train accuracy =  tensor(0.6888, device='cuda:0')
val accuracy =  tensor(0.4690, device='cuda:0')


In [ ]:
torch.save(model_fe.state_dict(), '/content/drive/MyDrive/chet_models/fine-tuned-inception-adv-training-with-stability.pt')

###Test on Imagenet-a, Imagenet-o data

In [ ]:
!unzip drive/MyDrive/182-cv-project/data/imagenet-a.zip
imagenet_a_path = pathlib.Path('imagenet-a/')
!unzip drive/MyDrive/182-cv-project/data/imagenet-o.zip
imagenet_o_path = pathlib.Path('imagenet-o/')

In [ ]:
class AdversarialDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep='\t', names=['image', 'label', 'x1', 'y1', 'x2', 'y2'])
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = Image.open(img_path)
    image = image.convert('RGB')
    label = self.img_labels.iloc[idx, 1]
    label = np.where(label==CLASS_NAMES)[0][0]
    if self.transform:
        image = self.transform(image)
    return image, label